<a href="https://colab.research.google.com/github/tharusha20241825-afk/MLCW/blob/main/Tharusha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [3]:
# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

In [4]:
# LOAD DATA
file_path = '/content/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(file_path)

print("=" * 60)
print("EXPLORATORY DATA ANALYSIS - TELCO CUSTOMER CHURN")
print("=" * 60)

EXPLORATORY DATA ANALYSIS - TELCO CUSTOMER CHURN
